## 載入套件

In [13]:
# predict
import os,shutil,sys#建資料夾用
import time #sleep用
import argparse
import json
import cv2
from utils.utils import get_yolo_boxes, makedirs
from utils.bbox import draw_boxes
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import math
import pandas as pd
from keras.models import load_model#載入h5檔案使用
import tensorflow as tf
import glob

## 初始設定載入

In [11]:
'''
首先需要一個training後的.h5結構權重檔案
參考權重檔(另一種載入方法)
https://stackoverflow.com/questions/53295570/userwarning-no-training-configuration-found-in-save-file-the-model-was-not-c
'''
infer_model = load_model('H:\\predict_model\\imt_yolo3.h5',compile=False)

In [14]:
#建立資料夾
def emtydir(dirname):
    if os.path.isdir(dirname):
        shutil.rmtree(dirname)
        time.sleep(2)
    os.mkdir(dirname)
#測試資料夾
def load_dataset(path):
    try:
        file=glob.glob(path+'*.jpg')
        count=0
        for i,f in enumerate(file):
            count=count+i
        print("路徑正常有",i,"個影像")
    except:
        print("檔案問題")

In [15]:
#輸入進來的測試路徑
input_path   = 'H:\\test\\'
#標框結果存檔路徑
output_path  = 'H:\\Result\\'

#新建Folder
emtydir(output_path)
load_dataset('H:\\test\\')
#輸出路徑正常有15個檔案

### Anchors設置(不需要)

In [ ]:
ROOT_PATH = 'H:/predict_model/'
# 數量
CLUSTERS = 6
# 影像尺寸
SIZE = 640

# 格式
def load_dataset(path):
    jpegimages = os.path.join(path, 'JPEGImages')
    if not os.path.exists(jpegimages):
        print('沒照片')
        sys.exit(0)

    label_file = os.listdir(labels_txt)
    print('label count: {}'.format(len(label_file)))
    dataset = []

    for label in label_file:
        with open(os.path.join(labels_txt, label), 'r') as f:
            txt_content = f.readlines()

        for line in txt_content:
            line_split = line.split(' ')
            roi_with = float(line_split[len(line_split)-2])
            roi_height = float(line_split[len(line_split)-1])
            if roi_with == 0 or roi_height == 0:
                continue
            dataset.append([roi_with, roi_height])
            # print([roi_with, roi_height])

    return np.array(dataset)

data = load_dataset(ROOT_PATH)
out = kmeans(data, k=CLUSTERS)

print(out)
print("Accuracy: {:.2f}%".format(avg_iou(data, out) * 100))
print("Boxes:\n {}-{}".format(out[:, 0] * SIZE, out[:, 1] * SIZE))

ratios = np.around(out[:, 0] / out[:, 1], decimals=2).tolist()
print("Ratios:\n {}".format(sorted(ratios)))

## 撈取BBOX的座標位置

In [ ]:
# Set some parameter
net_h, net_w = 608, 608 # a multiple of 32, the smaller the faster
#物件信心值、重疊度
obj_thresh,nms_thresh=0.75,0.45

anchors = [55,69, 75,234, 133,240, 136,129, 142,363, 203,290, 228,184, 285,359, 341,260]
#[0],[1],[2]照字母順序編排標籤順序
labels = ['Mark','X','Y'] 


# Read file
image_paths = []
if os.path.isdir(input_path): 
    for inp_file in os.listdir(input_path):
        image_paths += [input_path + inp_file]
else:
    image_paths += [input_path]

image_paths = [inp_file for inp_file in image_paths if (inp_file[-4:] in ['.jpg', '.png', '.JPG'])]

# the main loop
for image_path in image_paths:
    image = cv2.imread(image_path)
    print(image_path)

# predict the bounding boxes
    boxes = get_yolo_boxes(infer_model, [image], net_h, net_w, anchors, obj_thresh, nms_thresh)[0]

# draw bounding boxes on the image using labels
    draw_boxes(image, boxes, labels, obj_thresh)  

# write the image with bounding boxes to file
#     output_img_path = output_path + image_path.split('/')[-1]
    output_img_path = output_path +'predict_'+image_path.split('\\')[-1]
    cv2.imwrite(output_img_path, np.uint8(image))
    img=cv2.imread(output_img_path)[:,:,::-1]
    print(output_img_path)
#輸出
'''
H:\test\t001_03.jpg
H:\Result\predict_t001_03.jpg
H:\test\t001_04.jpg
H:\Result\predict_t001_04.jpg
H:\test\t002_03.jpg
H:\Result\predict_t002_03.jpg
H:\test\t002_04.jpg
H:\Result\predict_t002_04.jpg
H:\test\t003_03.jpg
H:\Result\predict_t003_03.jpg
H:\test\t003_04.jpg
H:\Result\predict_t003_04.jpg
H:\test\t004_03.jpg
H:\Result\predict_t004_03.jpg
H:\test\t004_04.jpg
H:\Result\predict_t004_04.jpg
H:\test\t005_03.jpg
H:\Result\predict_t005_03.jpg
H:\test\t005_04.jpg
H:\Result\predict_t005_04.jpg
H:\test\t006_03.jpg
H:\Result\predict_t006_03.jpg
H:\test\t006_04.jpg
H:\Result\predict_t006_04.jpg
H:\test\t007_03.jpg
H:\Result\predict_t007_03.jpg
H:\test\t007_04.jpg
H:\Result\predict_t007_04.jpg
H:\test\t008_03.jpg
H:\Result\predict_t008_03.jpg
H:\test\t008_04.jpg
H:\Result\predict_t008_04.jpg
'''

In [1]:
labs=[]
for m in range(len(boxes)):
    if boxes[m].score !=-1:
        label=boxes[m].label
        if label==0:
            label_name="Mark"
        elif label==1:
            label_name="X"
        elif label==2:
            label_name="Y"
        if(boxes[m].score>0.80):
            print(label_name,boxes[m].xmin,boxes[m].ymin,boxes[m].xmax,boxes[m].ymax,"評分",boxes[m].score)
#輸出
'''
X    1 -3 188 509 194
Y    2 234 29 251 291
Mark 0 244 158 327 192
X    1 -25 185 329 197
Y    2 237 113 251 432
'''

In [1]:
#將輸出轉為Pandas